In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shutil
from tqdm import tqdm
import cv2
from skimage.io import imread_collection
from skimage import exposure
import random
import glob
import torch
from PIL import Image

from matplotlib import pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
import zipfile
with zipfile.ZipFile('../input/platesv2/plates.zip', 'r') as zip_obj:
   # Extract all the contents of zip file in current directory
   zip_obj.extractall('/kaggle/working/')

In [3]:
print('After zip extraction:')
print(os.listdir("/kaggle/working/"))


data_root = '/kaggle/working/plates/'
print(os.listdir(data_root))

In [4]:
train_dir = 'train'
val_dir = 'val'

data_root = '/kaggle/working/plates/'

class_names = ['cleaned', 'dirty']

for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)
        
        
for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if i % 6 != 0:
            dest_dir = os.path.join(train_dir, class_name) 
        else:
            dest_dir = os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

In [5]:
images_train_0 = imread_collection('/kaggle/working/train/cleaned/*.jpg')
images_train_1 = imread_collection('/kaggle/working/val/dirty/*.jpg')
len(images_train_0), len(images_train_1)

In [6]:
# im_example = Image.open('/kaggle/working/train/cleaned/0009.jpg')

# for ex in glob.glob('/kaggle/working/train/cleaned/' + '*.jpg'):
#     im_example = cv2.imread(ex)
#     zoom_im = zoom(im_example, 0.7)
#     plt.imshow(zoom_im, interpolation = 'nearest')
#     plt.show()

In [7]:
def rotate_image(im, angle):
    
    image_center = tuple(np.array(im.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(im, rot_mat, im.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result
 
    
def fill(img, h, w):
    img = cv2.resize(img, (h, w), cv2.INTER_CUBIC)
    return img


def zoom(im, value):
    
    
    value = random.uniform(value, 1)
    h, w, _ = im.shape
    h_taken = int(value * h)
    w_taken = int(value * w)
    
    h_start = random.randint(0, h - h_taken)
    w_start = random.randint(0 , w - w_taken)
    
    img = im[h_start:h_start+h_taken, w_start:w_start+w_taken, :]
    img = fill(img, h, w)
    
    return img 


def data_augmentation(files_path, dataset_dir, class_dir):
    
    i = len(files_path)
    
    while (i > 0):
        
        path = files_path[i - 1]
        image = cv2.imread(path)
        
        for angle in range(90, 360, 45):
            rot = rotate_image(image, angle)
            cv2.imwrite(f'/kaggle/working/{dataset_dir}/{class_dir}/000{i}rotate_{angle}.jpg', rot)
            
        for zoom_ratio in np.linspace(0.7, 0.9, 5, endpoint=True):
            zoom_im = zoom(image, zoom_ratio)
            cv2.imwrite(f'/kaggle/working/{dataset_dir}/{class_dir}/000{i}zoomed_{zoom_ratio}.jpg', zoom_im)
        
        
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        flip = cv2.flip(image, 1)
        
        cv2.imwrite(f'/kaggle/working/{dataset_dir}/{class_dir}/000{i}rgb.jpg', rgb)
        cv2.imwrite(f'/kaggle/working/{dataset_dir}/{class_dir}/000{i}flip.jpg', flip)
        
        i -= 1
        
        
def background_removal(files_path):
    
    for file_path in files_path:
        #Load the Image
        imgo = cv2.imread(file_path)
        height, width = imgo.shape[:2]

        #Create a mask holder
        mask = np.zeros(imgo.shape[:2],np.uint8)

        #Grab Cut the object
        bgdModel = np.zeros((1,65),np.float64)
        fgdModel = np.zeros((1,65),np.float64)

        #Hard Coding the Rect The object must lie within this rect.
        rect = (10,10,width-30,height-30)
        cv2.grabCut(imgo,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
        mask = np.where((mask==2)|(mask==0),0,1).astype('uint8')
        img1 = imgo*mask[:,:,np.newaxis]

        #Get the background
        background = imgo - img1

        #Change all pixels in the background that are not black to white
        background[np.where((background > [0,0,0]).all(axis = 2))] = [255,255,255]

        #Add the background and the image
        final = background + img1

        cv2.imwrite(file_path, final)

def get_name_file(path_dir):
    for root, dirs, files in os.walk(path_dir):
        files = [path_dir + f for f in files if not f[0] == '.']
    return files
        
dataset_dir = ['train', 'val']
class_names = ['cleaned', 'dirty']

# /kaggle/working/train/cleaned/*.jpg


for dataset in dataset_dir:
    for class_ in class_names:
        files = get_name_file(f'/kaggle/working/'+ dataset + '/' + class_ + '/')
        background_removal(files)
        data_augmentation(files, dataset, class_)

In [8]:
%ls /kaggle/working/train/cleaned/*.jpg | wc -l
%ls /kaggle/working/train/dirty/*.jpg | wc -l

%ls /kaggle/working/val/cleaned/*.jpg | wc -l
%ls /kaggle/working/val/dirty/*.jpg | wc -l

In [9]:
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader 
from torchvision import models
import torchvision.transforms as tt
import torch.nn as nn

import torch.optim as optim


In [10]:
train_dir = 'train'
val_dir = 'val'

train_transforms = tt.Compose([
    tt.CenterCrop(224),
#     tt.RandomHorizontalFlip(),
#     tt.RandomVerticalFlip(),
    tt.ToTensor(),
    tt.Lambda(lambda x: x[np.random.permutation(3), :, :]),
    tt.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = tt.Compose([
    tt.CenterCrop(224),
    tt.ToTensor(),
    tt.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


train_dataset = ImageFolder(train_dir, train_transforms)
val_dataset = ImageFolder(val_dir, val_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2)

len(train_dataloader), len(train_dataset)

In [11]:
# def CalcMeanAndSTD(dataset):
    
#     loader = DataLoader(dataset, 
#                        batch_size = 10,
#                        num_workers = 0,
#                        shuffle = False)
    
#     mean = 0.0
#     std = 0.0
#     nb_samples = 0.0
    
#     for data, label in loader:
#         batch_samples = data.size(0)
#         # Rearrange batch to be the shape of [B, C, W * H]
#         data = data.view(batch_samples, data.size(1), -1)
        
#         mean += data.mean(2).sum(0)
#         std += data.std(2).sum(0)
#         nb_samples += batch_samples
    
#     mean /= nb_samples
#     std /= nb_samples
    
#     return mean, std

# mean_val, std_val = CalcMeanAndSTD(train_dataset)


# norm = tt.Normalize(mean = mean_val, std = std_val)

# train_ds = norm(train_dataset)

# val_ds = ImageFolder(val_dir, transform = tt.Compose([
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean = mean_val, std = std_val)
# ]))

In [12]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

def show_input(input_tensor, title = ''):
    image = input_tensor.permute(1,2,0).numpy()
    
    image = std * image + mean
    plt.imshow(image.clip(0, 1))
    plt.title(title)
    plt.show()
    plt.pause(0.001)

X_batch, y_batch = next(iter(train_dataloader))

for x_item, y_item in zip(X_batch, y_batch):
    show_input(x_item, title=class_names[y_item])

In [13]:
seed = 21
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

model = models.resnet50(pretrained=True)

# Disable grad for all conv layers
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(nn.Dropout(0.4),nn.Linear(model.fc.in_features, 64),nn.ReLU(),
                         nn.BatchNorm1d(num_features=64),nn.Dropout(0.3), nn.Linear(64,2))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

loss = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), amsgrad = True, lr = 1e-3)

# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
    factor=0.1, patience=10, threshold=0.0001, threshold_mode='abs')


In [14]:
def train_model(model, loss, optimizer, scheduler, num_epochs):
        
    loss_train = []
    acc_train = []
    
    loss_val = []
    acc_val = []
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_dataloader

                # Установить модель в режим обучения
                model.train() 
            else:
                dataloader = val_dataloader
                # Установить модель в режим оценки
                model.eval()

            running_loss = 0.
            running_acc = 0.

            for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim=1)

                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()

                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean().cpu()

            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)
            
            if phase == 'val':
                scheduler.step(epoch_loss)
                
#             scheduler.step()
            
            if phase == 'train':
                loss_train.append(epoch_loss)
                acc_train.append(epoch_acc)
            else:
                loss_val.append(epoch_loss)
                acc_val.append(epoch_acc)
                print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)

    return loss_train, acc_train, loss_val, acc_val

In [15]:
loss_train = {}
acc_train = {}
loss_val = {}
acc_val = {}

loss_train['loss_train'], acc_train['acc_train'], loss_val['loss_val'], acc_val['acc_val'] = \
train_model(model, loss, optimizer, scheduler, num_epochs=60)

In [16]:
import seaborn as sns

sns.set_style('darkgrid')

sns.set(rc={'figure.figsize':(15, 10)})


def acc_loss_graph(losses_train, acc_train, losses_val, acc_val, save_file_name='plot.png', download=False):
    
    fig, (ax1, ax2) = plt.subplots(1, 2)
    
    for experiment_id in losses_train.keys():
        ax1.plot(losses_train[experiment_id], label = experiment_id, c = 'red')
        
        
    for experiment_id in acc_train.keys():
        ax1.plot(acc_train[experiment_id], label = experiment_id, c = 'blue')
    
    ax1.legend()
    
    ax1.set_title('Accuracy')
    
    fig.tight_layout()
    
    
    for experiment_id in losses_val.keys():
        ax2.plot(losses_val[experiment_id], label = experiment_id, c = 'red')
        
        
    for experiment_id in acc_val.keys():
        ax2.plot(acc_val[experiment_id], label = experiment_id, c = 'blue')
    
    ax2.legend()
    ax2.set_title('Losses')

    
    fig.tight_layout()
    
    if download:
        fig.savefig(save_file_name)

In [17]:
acc_loss_graph(loss_train, acc_train, loss_val , acc_val)

In [18]:
test_dir = 'test'
shutil.copytree(os.path.join(data_root, 'test'), os.path.join(test_dir, 'unknown'))

In [19]:
#Мы не знаем, какие ID, какие названия изображения у нас генерируется, когда мы просим у DataLoader -- "дай нам следующий батч".
#Они по алфавиту идут, по дате создания, или просто случайным образом -- непонятно.
#Поэтому нам нужно переписать немножко ImageFolder, чтобы он нам отдавал не просто tuple, с самим изображением и его меткой, а ещё, чтобы он отдавал имя, ну, либо -- путь к изображению.

class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path
    
test_dataset = ImageFolderWithPaths('/kaggle/working/test', val_transforms)

batch_size = 15

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

    
model.eval() #Переводим модель в состояние eval
test_predictions = []  #Создаем пустой список предсказания 
test_img_paths = [] #Пути до изображения
for inputs, labels, paths in tqdm(test_dataloader): #Цикл по test_dataloader inputs - батч с изображением, lable - тут none, paths - пути до изображения  
    inputs = inputs.to(device) 
    labels = labels.to(device)  
    with torch.set_grad_enabled(False):
        preds = model(inputs) # Считаем предикшены
    test_predictions.append(
        torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy()) #С помощью torch.nn.functional.softmax получаем вероятности, для первого класса [:,1], пеереводим тензор в .data, на .cpu(), в numpy 
    test_img_paths.extend(paths)
test_predictions = np.concatenate(test_predictions)


In [20]:
inputs, labels, paths = next(iter(test_dataloader))

for img, pred in zip(inputs, test_predictions):
    show_input(img, title=pred)

In [21]:
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})
submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.8 else 'cleaned')
submission_df['id'] = submission_df['id'].str.replace('/kaggle/working/test/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)
submission_df

In [22]:
submission_df.to_csv('submission.csv')

In [23]:
# !rm -rf train val test